In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
from joblib import dump, load

module_path = os.path.abspath(os.path.join('..', 'code'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import utils
import numpy as np

/Users/german.moreno/Downloads/Thesis-SmartCampusVisualization/backend/code/utils.py:164: SyntaxWarning: invalid escape sequence '\.'
  data = data[~data['Edificio'].str.contains('\.')]


## Procesado de datos Oferta de cursos

In [5]:
df = utils.load_data('../data/courses.csv')

In [8]:
def initial_filter(df):
    df = df[df['Lugar']=='CAMPUS PRINCIPAL']
    # Agregar columna Edificio a los cursos de deporte que no lo tengan
    df['Edificio'] = df.apply(lambda x:'.Edif. Gata Golosa(GA)' if  x['Curso'].startswith('DEPO') else x['Edificio'],axis=1)
    df['Salón'] = df.apply(lambda x:'GA' if  x['Curso'].startswith('DEPO') else x['Salón'],axis=1)
    # Eliminar filas duplicadas
    df.drop_duplicates(inplace=True)
    
    return df

In [9]:
def further_processing(df):
    # Mantener solo las columnas necesarias
    df = df[['Curso','NRC','Periodo','Edificio','Salón','Días','Horas', 'Inscritos']]
    # Eliminar cursos sin inscritos
    df = df[df['Inscritos']>0]
    return df

In [10]:
def save_file(processed_df, path):
    processed_df.to_csv(path, index=False)

In [11]:
def process_file(input_path, output_path):
    df = utils.load_data(input_path)
    df = initial_filter(df)
    df = further_processing(df)
    save_file(df, output_path)

In [109]:
dump(process_file, '../models/process_file.joblib')

['../models/process_file.joblib']

In [12]:
process_file_loaded = load('../models/process_file.joblib')

process_file_loaded('../data/courses.csv', '../data/processed_courses.csv')



/var/folders/53/mgt9jd3n5qb5qsmyp0ltjdfm0000gp/T/ipykernel_38399/2259830601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Edificio'] = df.apply(lambda x:'.Edif. Gata Golosa(GA)' if  x['Curso'].startswith('DEPO') else x['Edificio'],axis=1)
/var/folders/53/mgt9jd3n5qb5qsmyp0ltjdfm0000gp/T/ipykernel_38399/2259830601.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Salón'] = df.apply(lambda x:'GA' if  x['Curso'].startswith('DEPO') else x['Salón'],axis=1)
/var/folders/53/mgt9jd3n5qb5qsmyp0ltjdfm

### Usando los datos de talanqueras

In [4]:
import pandas as pd

In [5]:
courses = pd.read_csv('../data/processed_courses.csv')
talanqueras = pd.read_csv('../data/output.csv')

In [5]:
courses.shape

(6935, 8)

In [6]:
talanqueras['Edificio'] = talanqueras['Puerta'].str.split('-', expand=True)[0]
talanqueras['Edificio'] = talanqueras['Edificio'].str.replace(r'[0-9]', '', regex=True)

courses['Salón'] = courses['Salón'].str.replace('.','')
courses['Edificio'] = courses['Salón'].str.split('_', expand=True)[0]

In [13]:
courses.sample(10)

NameError: name 'courses' is not defined

In [6]:
def count_entradas(edificio):
    return talanqueras['Edificio'].str.contains(edificio).sum()

In [7]:
edif = pd.DataFrame()
edif['Edificio'] = courses['Edificio'].unique()
for edificio in edif['Edificio']:
    edif.loc[edif['Edificio']==edificio, 'Entradas'] = count_entradas(edificio)
edif.head()  #  Conteo de entradas por edificio

,Edificio,Entradas
0,O,0.0
1,RGD,10088.0
2,ML,35062.0
3,AU,0.0
4,SD,35198.0


In [8]:
# df para contar entradas por rangos de horas
hours_count = pd.DataFrame()
hours_count['Edificio'] = courses['Edificio']
hours_count['Rango de Horas'] = talanqueras['FechaHora'].str.split(' ', expand=True)[1].str.split(':', expand=True)[0].astype(float) 

In [9]:
hours_count.head()

,Edificio,Rango de Horas
0,O,7.0
1,O,6.0
2,O,7.0
3,O,12.0
4,O,15.0


In [44]:
import numpy as np

def create_ranges(df):
    earliest = float(hours_count['Rango de Horas'].min())
    latest = float(hours_count['Rango de Horas'].max())
    df['Rango de Horas'] = pd.cut(df['Rango de Horas'], bins=np.arange(earliest, latest, 0.5), right=False)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

create_ranges(hours_count)

,Edificio,Rango de Horas
0,RGD,"[7.0, 7.5)"
1,RGD,"[6.0, 6.5)"
2,ML,"[7.0, 7.5)"
3,ML,"[12.0, 12.5)"
4,RGD,"[15.0, 15.5)"
...,...,...
86,ML,"[4.0, 4.5)"
87,LL,"[2.0, 2.5)"
88,RGD,"[2.0, 2.5)"
89,LL,"[1.0, 1.5)"


In [45]:
#count entries by hour
hours_count['Entradas'] = 0
for i in range(len(hours_count)):
    edificio = hours_count['Edificio'].iloc[i]
    left = hours_count['Rango de Horas'].iloc[i].left
    right = hours_count['Rango de Horas'].iloc[i].right
    for j in range(len(talanqueras)):
        hora = int(talanqueras['FechaHora'].iloc[j].split(' ')[1].split(':')[0])
        building = talanqueras['Edificio'].iloc[j]
        if (hora == left or hora == right) and building == edificio:
            hours_count['Entradas'].iloc[i] += 1

C:\Users\carlo\AppData\Local\Temp\ipykernel_13516\2067999010.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hours_count['Entradas'].iloc[i] += 1


In [46]:
#Ordenar por edificio y rango de horas
hours_count = hours_count.sort_values(by=["Edificio", "Rango de Horas"])

In [47]:
hours_count

,Edificio,Rango de Horas,Entradas
90,LL,"[0.0, 0.5)",22
89,LL,"[1.0, 1.5)",34
87,LL,"[2.0, 2.5)",27
68,LL,"[3.0, 3.5)",26
83,LL,"[4.0, 4.5)",25
...,...,...,...
35,SD,"[18.0, 18.5)",1043
50,SD,"[19.0, 19.5)",1087
30,SD,"[20.0, 20.5)",957
16,SD,"[21.0, 21.5)",681


In [48]:
# Exportamos los 2 df a csv
edif.to_csv('../data/entradas_edificio.csv', index=False)
hours_count.to_csv('../data/entradas_edificio_hora.csv', index=False)

## Agrupacion de datos usando oferta de cursos

In [7]:
new = courses.copy()

In [8]:
new.sample(10)

,Curso,NRC,Periodo,Edificio,Salón,Días,Horas,Inscritos
6367,ISIS-1404 TI EN LAS ORGANIZACIONES (TIPO E),16847,202420,TX,TX_602,M,1530-1650,22
4642,FISI-1518P COMPLEMENTARIA Y LABORATORIO FISICA I,72734,202420,C,C_106,L,1100-1220,22
602,"ARQT-2104 UNID.NÚC.PROY, DE LA FORMA DE LA ARQ...",72628,202420,C,C_306,M,0700-0950,10
5327,ICYA-4138 MANEJO SOSTENIBLE DEL AGUA URBANA,71233,202420,RGD,RGD_211,L,1230-1350,11
1977,CISO-4606 MAPAS Y ANALISIS CON SIG,70791,202420,GB,GB_206,V,0930-1220,8
2048,CISO-4842 SEMINARIO SOCIOLOGÍA RURAL,70796,202420,VIRT,VIRT,S,0800-1050,9
7192,LENG-1159 ENGLISH 09A WRITING 2 - FOR SCIENCE ...,65937,202420,SD,SD_307,I,0800-0920,17
2959,DERE-3302 ACCIONES PUBLICAS,10051,202420,RGD,RGD_306-7,L,0800-0920,31
8832,MECA-4108C COMPLEMENTARIA MICROECONOMETRIA APL,70591,202420,SD,SD_401,S,0800-0920,19
95,ADMI-1412 CONTABILIDAD FINANCIERA,72603,202420,W,W_403,J,0800-0920,39


In [9]:
new.drop(columns=['Periodo','Salón'], inplace=True)
new.dropna(inplace=True)

In [10]:
#Se filtran los cursos que tienen clase los miercoles (I)
new = new[new['Días'].str.contains('I')]

In [11]:
new.shape

(1837, 6)

In [12]:
new.sort_values(by=['Edificio','Horas'], inplace=True)
new.drop_duplicates( keep='first', inplace=True)
new.reset_index(drop=True, inplace=True)

In [13]:
def convert_to_range(horas):
    horas = str(horas)
    # Split the start and end times
    start, end = horas.split("-")

    start_hour = int(start[:2])  # First two digits (hours)
    end_hour = int(end[:2])      # First two digits (hours)
    if end[2:] != '00':
        end_hour += 1

    return[start_hour, end_hour]

# Apply the conversion function
new["Rango de Horas"] = new["Horas"].apply(convert_to_range)

In [14]:
new.drop(columns=['Horas'], inplace=True)
new['Rango de Horas'] = new['Rango de Horas'].apply(tuple)

In [15]:
#Nos da el numero de estudiantes inscritos en un curso por edificio
inscritos = new.groupby('Edificio', as_index=False)['Inscritos'].sum()

In [16]:
new

,Curso,NRC,Edificio,Días,Inscritos,Rango de Horas
0,DERE-2207 ARGUMENTACION EN PROCESOS CIVILES,68762,,I,20,"(9, 11)"
1,DERE-2206 PROPIEDAD Y DERECHOS REALES,20376,AU,I,37,"(6, 8)"
2,ISIS-1404 TI EN LAS ORGANIZACIONES (TIPO E),12125,AU,I,22,"(6, 8)"
3,ISIS-2503 ARQUITECTURA Y DISEÑO DE SOFTWARE,63026,AU,I,31,"(6, 8)"
4,LENG-1157 ENGLISH 07 - SPEAKING 1 (CICLO 1 DE ...,27929,AU,I,21,"(6, 8)"
...,...,...,...,...,...,...
1673,CPER-2401 NARRATIVA CORPORAL 1 (CICLO 1 DE 8 S...,57272,Z,I,10,"(12, 14)"
1674,IBIO-1110 PROGRAMA DE ACOMPAÑAMIENTO,42304,Z,I,25,"(12, 14)"
1675,GEOC-3606 GEODINÁMICA,62646,Z,I,12,"(14, 16)"
1676,ICYA-1125L LABORATORIO DE GEOMÁTICA,13126,Z,I,25,"(15, 17)"


In [17]:
# Vamos a generar todos los rangos de horas posibles por periodos de media hora
all_ranges = []
earliest = new['Rango de Horas'].apply(lambda x: x[0]).min()
latest = new['Rango de Horas'].apply(lambda x: x[1]).max()
current = earliest
while current < latest:
    all_ranges.append((current, current + 0.5))
    current += 0.5

In [18]:
def count_missing_ranges(start, end, ranges):
    # Count the number of missing ranges
    count = 0
    start_index = 0
    end_index = 0
    for r in ranges:
        if r[0] == start:
            start_index = ranges.index(r)
        if r[1] == end:
            end_index = ranges.index(r)
    count = end_index - start_index + 1
    return (count, start_index, end_index)

def create_row(df, row, start_index, end_index, ranges):
    # Create new rows for the missing ranges and add them to a list
    new_rows = []
    for i in range(start_index, end_index + 1):
        new_row = row.copy()
        new_row['Rango de Horas'] = ranges[i]
        new_row['Inscritos'] = row["Inscritos"]  
        new_rows.append(new_row)  # Append the row to the list
    
    # Concatenate the new rows to the DataFrame at once
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    return df
    

In [19]:
#Se ajustan los rangos de horas con respecto a los que se acaban de generar
for row in new.itertuples():
    start = row[6][0]  # Extract start time from 'Rango de Horas'
    end = row[6][1]    # Extract end time from 'Rango de Horas'
    
    count, start_index, end_index = count_missing_ranges(start, end, all_ranges)
    # If there are missing ranges, add new rows
    if count > 1:
        new = create_row(new, row._asdict(), start_index, end_index, all_ranges)

In [20]:
new.groupby('Edificio', as_index=False)['Inscritos'].sum()

,Edificio,Inscritos
0,,100
1,AU,15839
2,B,10003
3,C,21729
4,CJ,315
5,G,1355
6,GA,2676
7,IP,10
8,J,1770
9,K2,293


In [21]:
#Se borran las columnas que no se necesitan y se filtran los cursos que tienen un rango de horas por cada media hora
#new.drop(columns=['Index','_6'], inplace=True)
filtered_df = new[new['Rango de Horas'].apply(lambda x: isinstance(x[0], float) and isinstance(x[1], float))]
lista_edif = ['SD','RGD','ML','LL','O','C','AU','GA','TX']
filtered_df = filtered_df[filtered_df['Edificio'].isin(lista_edif)]
filtered_df.reset_index(drop=True, inplace=True)

In [28]:
filtered_df

,Edificio,Rango de Horas,Inscritos
0,AU,"(6.5, 7.0)",310
1,AU,"(7.0, 7.5)",507
2,AU,"(7.5, 8.0)",507
3,AU,"(8.0, 8.5)",562
4,AU,"(8.5, 9.0)",562
...,...,...,...
243,TX,"(18.5, 19.0)",8
244,TX,"(19.0, 19.5)",8
245,TX,"(19.5, 20.0)",8
246,TX,"(20.0, 20.5)",5


In [157]:
filtered_df.to_csv('../data/entradas_oferta.csv', index=False)

In [27]:
filtered_df = filtered_df.groupby(['Edificio', 'Rango de Horas'], as_index=False)['Inscritos'].sum()

In [159]:
filtered_df.to_csv('../data/entradas_edificio_hora_completos.csv', index=False)

## Para llenar los rangos faltantes

In [84]:
inscritos.to_csv('../data/inscritos_edificio.csv')

In [63]:
import pandas as pd
import numpy as np


df = pd.read_csv('../data/entradas_edificio_hora.csv')

# Convert "Rango de Horas" to numeric ranges for easier processing
def parse_range(range_str):
    range_str = range_str.replace("(", "[").replace(")", "]").strip()
    start, end = map(float, range_str.strip("[]").split(", "))
    return start, end

df[["Start", "End"]] = df["Rango de Horas"].apply(parse_range).apply(pd.Series)

# Initialize an empty DataFrame to store results
all_ranges = []

# Process each unique "Edificio" separately
for edificio in df["Edificio"].unique():
    # Filter data for the current building
    edificio_df = df[df["Edificio"] == edificio]

    # Generate a complete range of intervals for this building
    start_min = edificio_df["Start"].min()
    end_max = edificio_df["End"].max()
    interval = 0.5
    complete_ranges = pd.DataFrame({
        "Start": np.arange(start_min, end_max, interval),
        "End": np.arange(start_min + interval, end_max + interval, interval),
        "Edificio": edificio  # Add the building identifier
    })

    # Merge with the filtered DataFrame
    merged = complete_ranges.merge(edificio_df, on=["Start", "End"], how="left")

    # Drop the redundant "Edificio_x" or "Edificio_y" columns and keep the correct one
    merged["Edificio"] = merged["Edificio_x"].fillna(merged["Edificio_y"])  # Select correct column
    merged.drop(columns=["Edificio_x", "Edificio_y"], inplace=True)  # Drop the redundant columns

    # Fill missing "Entradas" with 0
    merged["Entradas"].fillna(0, inplace=True)

    # Add to the results
    all_ranges.append(merged)

# Combine results from all buildings
final_df = pd.concat(all_ranges, ignore_index=True)

# Recreate "Rango de Horas" column for consistency
final_df["Rango de Horas"] = "[" + final_df["Start"].astype(str) + ", " + final_df["End"].astype(str) + "]"

# Drop temporary columns
final_df = final_df[["Edificio", "Rango de Horas", "Entradas"]]

# Sort and display the result
final_df = final_df.sort_values(by=["Edificio", "Rango de Horas"])
final_df.reset_index(drop=True, inplace=True)

In [94]:
for edificio in final_df['Edificio'].unique():
    # Filter rows where 'Edificio' matches and 'Entradas' is 0
    condition = (final_df['Edificio'] == edificio) & (final_df['Entradas'] == 0)
    
    #sacamos el indice que contiene un 0
    index = final_df.loc[condition].index
    
    for idx in index:
        if idx > 0 and idx < len(final_df) - 1:
            anterior = final_df.loc[idx - 1, 'Entradas']
            posterior = final_df.loc[idx + 1, 'Entradas']
            # Compute the mean for the current 'Edificio', ignoring zeros
            mean_value = np.mean([anterior, posterior])
            # Assign the mean value to rows where 'Entradas' is 0
            final_df.loc[idx, 'Entradas'] = mean_value


In [98]:
final_df

,Edificio,Rango de Horas,Entradas
0,LL,"[0.0, 0.5]",22.0
1,LL,"[0.5, 1.0]",0.0
2,LL,"[1.0, 1.5]",34.0
3,LL,"[1.5, 2.0]",0.0
4,LL,"[10.0, 10.5]",669.0
...,...,...,...
175,SD,"[7.5, 8.0]",0.0
176,SD,"[8.0, 8.5]",2327.0
177,SD,"[8.5, 9.0]",0.0
178,SD,"[9.0, 9.5]",2442.0


In [96]:
final_df.to_csv('../data/entradas_edificio_hora_completos.csv', index=False)

## Merge con datos de los AP

In [29]:
aps = pd.read_csv('../data/AP_data.csv', delimiter=';')
edificios = filtered_df.copy()

In [30]:
aps.sample(10)

,Bloque,Salon,Capacidad,Dispositivos conectados,Nombre AP,MAC,Hora
187,ML,ML_511,61,24.0,AP-652,02:65:11:18:d9:48,2024/10/01 15:34:16
222,O,O_101,121,117.0,AP-441,aa:99:22:45:1b:39,2024/10/01 09:43:51
43,C,C_307,60,NaN,NaN,NaN,NaN
186,ML,ML_510,69,30.0,AP-174,68:10:5d:bb:a3:dc,2024/10/01 22:58:42
22,Au,Au_307,31,14.0,AP-953,da:df:3d:25:ea:1a,2024/10/01 08:37:33
53,RGD,Rgd_0003,145,NaN,NaN,NaN,NaN
19,Au,Au_304,28,18.0,AP-262,50:52:5e:f3:67:2c,2024/10/01 19:55:59
244,O,O_205,62,18.0,AP-592,16:0a:f9:71:84:60,2024/10/01 14:43:43
104,ML,ML_113,10,8.0,AP-287,64:2c:db:3d:fe:88,2024/10/01 07:21:34
182,LL,LL_304,50,8.0,AP-131,70:d5:88:4a:a1:5b,2024/10/01 12:04:39


In [31]:
aps.dropna(inplace=True)

In [32]:
edificios

,Edificio,Rango de Horas,Inscritos
0,AU,"(6.5, 7.0)",310
1,AU,"(7.0, 7.5)",507
2,AU,"(7.5, 8.0)",507
3,AU,"(8.0, 8.5)",562
4,AU,"(8.5, 9.0)",562
...,...,...,...
243,TX,"(18.5, 19.0)",8
244,TX,"(19.0, 19.5)",8
245,TX,"(19.5, 20.0)",8
246,TX,"(20.0, 20.5)",5


In [33]:
# Se eliminan las columnas que no se necesitan
aps.drop(columns=['MAC','Capacidad'], inplace=True)
# Se cambia el formato de la columna "Hora" para que sea compatible con el formato de la columna "Rango de Horas"
aps['Hora'] = aps['Hora'].str.split(' ', expand=True)[1]
# Se convierte la columna "Hora" a formato datetime
aps['Hora'] = pd.to_datetime(aps['Hora'], format='%H:%M:%S')

# Definir los límites de tiempo
inicio = pd.to_datetime('06:30').time()
fin = pd.to_datetime('22:00').time()

# Filtrar el DataFrame
aps = aps[(aps['Hora'].dt.time >= inicio) & (aps['Hora'].dt.time <= fin)]

# Formatear la hora para que no tenga segundos
aps['Hora'] = aps['Hora'].dt.strftime('%H:%M')

In [34]:
aps

,Bloque,Salon,Dispositivos conectados,Nombre AP,Hora
0,Au,Au_101,12.0,AP-664,06:36
1,Au,Au_102,6.0,AP-138,10:34
2,Au,Au_103,12.0,AP-284,11:00
4,Au,Au_107,15.0,AP-780,08:23
6,Au,Au_201,5.0,AP-093,08:29
...,...,...,...,...,...
255,O,O_402,4.0,AP-884,09:55
256,O,O_402,21.0,AP-816,14:30
257,O,O_403,14.0,AP-388,08:31
258,O,O_403,22.0,AP-446,17:51


In [35]:
# Se cambia el formato de la columna "Hora" para que sea compatible con el formato de la columna "Rango de Horas"
def convertHourToFloat(hour):
    hour = hour.split(':')
    if hour[1] <= '29':
        return float(hour[0] + '.0')
    else:
        return float(hour[0] + '.5')

aps['Hora'] = aps['Hora'].apply(convertHourToFloat)

In [36]:
aps.drop(columns='Nombre AP', inplace=True)

In [37]:
result = aps.groupby(['Bloque', 'Salon', 'Hora'], as_index=False)['Dispositivos conectados'].sum()

In [38]:
result

,Bloque,Salon,Hora,Dispositivos conectados
0,Au,Au_101,6.5,12.0
1,Au,Au_102,10.5,6.0
2,Au,Au_103,11.0,12.0
3,Au,Au_107,8.0,15.0
4,Au,Au_201,8.0,5.0
...,...,...,...,...
160,RGD,Rgd_308,7.0,16.0
161,RGD,Rgd_309,6.5,14.0
162,RGD,Rgd_312,15.0,4.0
163,RGD,Rgd_313,14.0,6.0


In [39]:
result['Bloque'] = result['Bloque'].str.replace('Au', 'AU')
result['Bloque'] = result['Bloque'].str.replace('Tx', 'TX')
result.rename(columns={'Bloque':'Edificio'}, inplace=True)

In [41]:
edificios['Rango de Horas'][0][0]

6.5

In [42]:
# Function to check if Hora is within Rango de Horas
def check_range(row):
    for _, r in edificios[edificios['Edificio'] == row['Edificio']].iterrows():
        if r['Rango de Horas'][0] <= row['Hora'] < r['Rango de Horas'][1]:
            return r['Rango de Horas']
    return None

# Apply the function to create a new column in df1
result['Rango de Horas'] = result.apply(check_range, axis=1)

# Now merge on Edificio and Rango de Horas
merged_df = pd.merge(result.dropna(), edificios, on=['Edificio', 'Rango de Horas'], how='inner')

In [44]:
merged_df.to_csv('../data/AP_con_oferta.csv', index=False)